# Gestion automatique des données 
## Projet Programmation Web
### Nahida BENHAFFAF - Margaux BRULIARD

Nous cherchons ici à récupérer les informations de plusieurs films contenues dans un fichiers d'extension csv fourni pour le projet.

Nous souhaitons extraire en plus des données de films:
   * Les catégories de films
   * les noms des acteurs/réalisateurs
afin de générer plus simplement les tables associées dans notre base de données
 
A l'issu de notre programme, nous souhaitons pouvoir insérer automatiquement les données trouvées dans notre base de données à l'aide de requêtes SQL

-----
### <font color="blue">Importation des modules Python</font>

In [1]:
import pandas as pd
import csv
import sqlite3 as bdd
from os.path import exists
from os import system
from random import randint

----
### <font color="blue">Extraction des données contenues dans film.csv</font>

In [2]:
reader = pd.read_csv("../../film.csv", sep=';', header=0)
extract = reader.to_dict(orient='split')

columns_name = extract['columns'] # nom des colonnes
datas = extract['data'][1:] # Les données sur les films en enlevant la 1ere ligne qui correspond au type de donnée

print("Nom des colonnes : ", columns_name)

Nom des colonnes :  ['Year', 'Length', 'Title', 'Subject', 'Actor', 'Actress', 'Director', 'Popularity', 'Awards', '*Image']


On sélectionne ensuite parmi l'ensemble des films une vingtaine d'entre eux

In [3]:
fl = 121 # premiere ligne sélectionnée
n = 22 # nbr de lignes selectionnées

train = list()
if (fl + n > len(datas)):
    print("ERREUR: Vous ne pouvez pas sélectionner ces lignes ...")
else:
    train = datas[fl:n+fl]
# ...

Aperçu des données:

In [4]:
years = list()
for f in train:
    years.append(f[0])
# ...
print("Dates de parution des films: ", list(set(years)))

Dates de parution des films:  ['1970', '1990', '1945', '1942', '1982', '1985', '1972', '1975', '1988', '1989', '1977', '1971', '1948', '1944', '1978', '1935']


----
### <font color="blue">Liste des catégories</font>

In [5]:
categories = list()
for film in datas:
    categories.append(film[3])
# ...
categories = list(set(categories))

# on supprime nan
for cat in categories:
    if type(cat) is not str:
        categories.remove(cat)
    # ...
# ...
categories.sort()
print("Liste des catégories ", categories)
print("nombre de catégories: ", len(categories))

Liste des catégories  ['Action', 'Adventure', 'Comedy', 'Crime', 'Drama', 'Fantasy', 'Horror', 'Music', 'Mystery', 'Romance', 'Science Fiction', 'Short', 'War', 'Western', 'Westerns']
nombre de catégories:  15


Une version française de la liste

In [6]:
fr_cat = ['Action', 'Aventure', 'Comédie', 'Policier', 'Drame', 'Fantastique', 'Horreur', 'Musical', 'Mystère',
         'Romance', 'Science Fiction', 'Court-Métrage', 'Guerre', 'Western']

# on créé un dictionnaire pour gérer les id lors de la création de la table
categories.remove('Westerns')

dic_cat = {}
ind = 1
for cat in categories:
    dic_cat[cat] = ind
    ind += 1
# ...

# on met le même id a Western et Westerns
dic_cat['Westerns'] = dic_cat['Western']

print("Verification du dictionnaire des categories: ", dic_cat)

Verification du dictionnaire des categories:  {'Action': 1, 'Adventure': 2, 'Comedy': 3, 'Crime': 4, 'Drama': 5, 'Fantasy': 6, 'Horror': 7, 'Music': 8, 'Mystery': 9, 'Romance': 10, 'Science Fiction': 11, 'Short': 12, 'War': 13, 'Western': 14, 'Westerns': 14}


----
### <font color="blue">Liste des acteurs/réalisateurs</font>

In [7]:
individus = list()
for film in train:
    individus += film[4:7]
# ...
individus = list(set(individus))

print("Liste des individus: ", individus)

Liste des individus:  [nan, 'Albaic�n, Rafael', 'Milius, John', 'DeMille, Cecil B.', 'Benatar, Pat', 'Lang, Fritz', 'Beller, Kathleen', 'Mastorakis, Nico', 'Drury, David', 'Bening, Annette', 'Bj�rk, Halvar', 'Robinson, Edward G.', 'Redgrave, Michael', 'Burns, George', 'Rosenberg, Stuart', 'Bennett, Julie', 'Berger, Senta', 'Beecher, Janet', 'Hauser, Wings', 'Wayne, John', 'Gould, Elliott', 'Rush, Richard', 'Belford, Christine', 'Bergen, Candice', 'Bergman, Ingrid', 'Nichols, Mike', 'Boyer, Charles', 'Newman, Paul', 'Blanc, Mel', 'Busey, Gary', 'De Niro, Robert', 'Bauer, Belinda', 'Flanders, Ed', 'Connery, Sean', 'Attenborough, Richard', 'Beck, Kimberly', 'Kingsley, Ben', 'Adolphson, Edvin', 'Nicholson, Jack', 'Taylor, Jud', 'Bennett, Joan', 'Ford, Greg', 'Hackman, Gene', 'Ford, Harrison', 'Pakula, Alan J.', 'Winkler, Irwin', 'Bedelia, Bonnie', 'Beals, Jennifer', 'Berenson, Marisa', 'Wenders, Wim', 'Bergman, Ingmar', 'Cukor, George', 'Cage, Nicolas', 'Belzer, Richard']


In [8]:
individus = individus[1:]

print("nombre d'individus: ", len(individus))
print([st.strip() for st in individus[0].split(',')])

nombre d'individus:  53
['Albaic�n', 'Rafael']


----
### <font color="blue">Construction de la Base de données</font>

In [9]:
connexion = bdd.connect("../database.db")

Mise en place des requêtes à l'aide d'un curseur

In [10]:
curseur = connexion.cursor()

----
### <font color='red'> ATTENTION : Nettoyage des tables </font>

In [11]:
question = input("Voulez vous supprimer les tables de travail (categories, individus, films) ? " \
                 "Les tables seront supprimées dans la BDD et les données irrécupérables [Y/n]")
if (question == "Y"):
    curseur.execute('''DROP TABLE forum''')
    curseur.execute('''DROP TABLE notes''')
    curseur.execute('''DROP TABLE distribution''')
    curseur.execute('''DROP TABLE films''')
    curseur.execute('''DROP TABLE individus;''')
    curseur.execute('''DROP TABLE categories''')
    curseur.execute('''DROP TABLE users''')
# ...

Voulez vous supprimer les tables de travail (categories, individus, films) ? Les tables seront supprimées dans la BDD et les données irrécupérables [Y/n]Y


In [12]:
creation_queries = list() # liste des requetes de creation de tables dans la BDD

------
### <font color="blue">Création de la table Catégories</font>

In [13]:
query = '''CREATE TABLE IF NOT EXISTS categories (
    id_categorie INTEGER PRIMARY KEY AUTOINCREMENT UNIQUE, 
    nom TEXT
);'''
creation_queries.append(query)

curseur.execute(query)
connexion.commit()


for i in range(len(fr_cat)):
    curseur.execute('''INSERT INTO categories (id_categorie, nom) VALUES (?, ?)''', (i+1, fr_cat[i]))
# ...
connexion.commit()  #Ne pas oublier de valider les modifications

### <font color="blue">Création de la table Individus</font>
On crée la table individus si elle n'existe pas déjà en posant dans l'ordre (id_individu, nom, prenom, pays, annee_naissance, url_photo)

In [14]:
query = '''CREATE TABLE IF NOT EXISTS individus (
    id_individu INTEGER PRIMARY KEY AUTOINCREMENT UNIQUE,
    nom TEXT NOT NULL,
    prenom TEXT NOT NULL,
    pays TEXT,
    annee INTEGER CHECK(annee > 0),
    photo TEXT
);'''
creation_queries.append(query)

curseur.execute(query)
connexion.commit()


for ind in individus:
    nom, prenom = [st.strip() for st in ind.split(',')]
    curseur.execute('''INSERT INTO individus (nom, prenom, pays, annee, photo) VALUES (?, ?, NULL, NULL, NULL)''', (nom, prenom))
# ...
connexion.commit()

On note que ce cette façon on obtient pour id_individu (qui est incrémenté automatique), id_individu = i + 1, pour i l'indice de l'individu dans la liste individus

### <font color="blue">Création de la table Films</font>

In [15]:
query = '''CREATE TABLE IF NOT EXISTS films (
    id_film INTEGER PRIMARY KEY AUTOINCREMENT UNIQUE,
    titre TEXT NOT NULL,
    duree TEXT,
    id_categorie INTEGER,
    annee INTEGER CHECK(annee > 0),
    pays TEXT,
    id_realisateur INTEGER,
    photo TEXT,
    CONSTRAINT FK_FILM_REAL FOREIGN KEY (id_realisateur) REFERENCES individus (id_individu)
);'''
creation_queries.append(query)
curseur.execute(query)
connexion.commit()

query = '''CREATE TABLE IF NOT EXISTS distribution(
    id_film INTEGER NOT NULL, 
    id_acteur INTEGER NOT NULL,
    CONSTRAINT PK_DISTRIBUTION PRIMARY KEY (id_film, id_acteur),
    CONSTRAINT FK_DISTRIBUTION_FILM FOREIGN KEY(id_film) REFERENCES films (id_film),
    CONSTRAINT FK_DISTRIBUTION_ACTEUR FOREIGN KEY (id_acteur) REFERENCES individus(id_individu)
);'''
creation_queries.append(query)
curseur.execute(query)

# on remplit les tables distribution et film en même temps
id_film = 1
for film in train:
    
    # on recupere l'id du réalisateur en le cherchant dans la liste
    id_real = 'NULL'
    if type(film[6]) is str:
        id_real = individus.index(film[6]) + 1
    
    
    # et de meme pour les id des acteurs indiques
    id_act1 = 'NULL'
    if type(film[4]) is str:
        id_act1 = individus.index(film[4]) + 1
    
    id_act2 = 'NULL'
    if type(film[5]) is str:
        id_act2 = individus.index(film[5]) + 1
    
    # et enfin l'id de la categorie
    id_cat = 'NULL'
    if type(film[3]) is str:
        # on cherche l'id correspondant dans le dictionnaire
        id_cat = dic_cat[film[3]]
    # ...
    
    annee = 'NULL'
    if type(film[0]) is str:
        annee = film[0]
    # ...
    
    duree = 'NULL'
    if type(film[1]) is str:
        duree = film[1]
    # ...
    
    curseur.execute('''INSERT INTO films (titre, duree, id_categorie, annee, pays, id_realisateur, photo)
    VALUES (?, ?, ?, ?, NULL, ?, NULL)''', (film[2], duree, id_cat, annee, id_real))
    
    if not (id_act1 == 'NULL'):
        curseur.execute('''INSERT INTO distribution (id_film, id_acteur) VALUES (?, ?)''', (id_film, id_act1))
    # ...
    
    if not (id_act2 == 'NULL'):
        curseur.execute('''INSERT INTO distribution (id_film, id_acteur) VALUES (?, ?)''', (id_film, id_act2))
    # ...
    
    id_film += 1
# ...
connexion.commit()

### Création des tables vides 

In [16]:
query = '''CREATE TABLE IF NOT EXISTS users(
    pseudo TEXT PRIMARY KEY,
    email TEXT UNIQUE NOT NULL,
    password TEXT NOT NULL
);'''

creation_queries.append(query)
curseur.execute(query)
connexion.commit()

query = '''CREATE TABLE IF NOT EXISTS notes (
    id_film INTEGER,
    id_user TEXT,
    note INTEGER CHECK(note BETWEEN 1 AND 5) NOT NULL,
    commentaire TEXT,
    CONSTRAINT PK_NOTES PRIMARY KEY (id_film, id_user),
    CONSTRAINT FK_NOTES_FILM FOREIGN KEY (id_film) REFERENCES films (id_film),
    CONSTRAINT FK_NOTES_USER FOREIGN KEY (id_user) REFERENCES users (pseudo)
);'''

creation_queries.append(query)
curseur.execute(query)
connexion.commit()

query = '''CREATE TABLE IF NOT EXISTS forum (
    id_msg INTEGER PRIMARY KEY AUTOINCREMENT,
    id_user TEXT NOT NULL,
    id_film INTEGER NOT NULL,
    id_msg_ans INTEGER,
    contenu TEXT,
    date TEXT,
    CONSTRAINT FK_FORUM_USER FOREIGN KEY (id_user) REFERENCES users (pseudo),
    CONSTRAINT FK_FORUM_FILM FOREIGN KEY (id_film) REFERENCES films (id_film),
    CONSTRAINT FK_FORUM_REPONSE FOREIGN KEY (id_msg_ans) REFERENCES forum (id_msg)
);'''

creation_queries.append(query)
curseur.execute(query)
connexion.commit()

### Test de la base de données créée:
Nous cherchons les informations sur le film 1:

In [17]:
print("Affichage des informations obtenues par le fichier .csv:\n", train[1])

Affichage des informations obtenues par le fichier .csv:
 ['1988', '96', 'Split Decisions', 'Drama', 'Hackman, Gene', 'Beals, Jennifer', 'Drury, David', '52', 'No', 'NicholasCage.png']


In [18]:
curseur.execute('''SELECT annee, duree, titre, categories.nom FROM films 
                INNER JOIN categories ON categories.id_categorie = films.id_categorie 
                WHERE id_film = 2''')
print(curseur.fetchone())

curseur.execute('''SELECT nom, prenom FROM individus 
                INNER JOIN distribution
                ON distribution.id_acteur = individus.id_individu
                WHERE id_film = 2''')
print(curseur.fetchall())

(1988, '96', 'Split Decisions', 'Drame')
[('Hackman', 'Gene'), ('Beals', 'Jennifer')]


### Ecriture d'un fichier creation.sql 

Le fichier contiendra toutes les requetes de création de la base de données. Pour une re-génération de la base de données utérieure

Elle sera sauvegardée dans le fichier save_database/creation.sql

In [19]:
creation_file = open("save_database/creation.sql", "w")

for q in creation_queries:
    creation_file.write(q + '\n')
# ...
creation_file.close()

----
### Remplissage des tables users, notes et forum



In [20]:
pseudos_to_add = ['mbruliard', 
                  'nbenhaffaf',
                  'ProgWebForever',
                  'toto',
                  'coeurDartichaud', 
                  'LuckyTheLuc', 
                  'LaVraieCelineDeVegas', 
                  'AliceAuPaysDesBretons',
                  'JeSuisPersonne',
                  'MbappéForever',
                  'PotterHead1',
                  'jujuCo1n',
                  'NotAwrinkle',
                  'LeRoiPopCorn12',
                  'GrosBolDeRiz <3'                  
                 ]

# pour toto on place titi
# pour ProgWebForever -> 'foughali'
mdp_to_add = ['$2y$10$vTkLce23BDKt0c1VHB3Uk.puoDFJZeafu3QgoNmjfJwkM2/oS2Ax2' for i in range(0, len(pseudos_to_add))]
mdp_to_add[2] = '$2y$10$S.hJ5XDIJJWvDyE4eQdvZOxmT/cLzENTkLweIbpSSfj6whuMJM7xu' 
mdp_to_add[3] = '$2y$10$x2haEkjRqsVqzUGeLfooquhn6zNnTT0WzBv4JlShH4UoURQifGfKW'


# on remplit la table users
for i in range(0, len(pseudos_to_add)):
    curseur.execute('''INSERT INTO users (pseudo, email, password)
    VALUES (?, ?, ?)''', (pseudos_to_add[i], pseudos_to_add[i].lower() + '@dodo.com', mdp_to_add[i]))
# ...
connexion.commit()


---
### Remplissage partiel de la table notes

On va récupérer la valeur `popularity` dans `films.csv`.

On tire aléatoirement les pseudos des utilisateurs qui noteront un film. Puis pour chacun d'entre eux on tire aléatoirement une note entre 1 et 5

In [21]:
id_film = 1
for film in train:
    
    for pseudo in pseudos_to_add:
        has_noted = randint(0,1)
        if has_noted :
            note = randint(1, 5)
            curseur.execute('''INSERT INTO notes (id_film, id_user, note, commentaire) VALUES (?, ?, ?, ?)''', 
                            (id_film, pseudo, note, ''))
        # ...
    # ...        
    connexion.commit()
    id_film += 1
# ...

### Générer des fichiers csv pour sauvegarder les données de la base de données

On génère la sauvegarde pour toutes les tables de la base de données à l'exception de la table <font color="green">utilisateurs</font> qui elle est construite manuellement et est donc sauvegardé par défaut

In [22]:
table_names = ['categories', 'individus', 'films', 'distribution', 'users', 'notes', 'forum']

for table in table_names:
    curseur.execute("SELECT * FROM " + table + ";")
    with open("save_database/save_" + table + ".csv", "w", newline='') as csv_file:
        csv_writer = csv.writer(csv_file, delimiter=';')
        csv_writer.writerow([i[0] for i in curseur.description]) # write headers
        csv_writer.writerows(curseur)
    # ...
# ...

Fermeture de la base de données et du fichier de création de la BDD

In [23]:
connexion.close()